In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar las bibliotecas necesarias
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Importar bibliotecas
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [ ]:
# Función para leer párrafos de los archivos
def leer_parrafos(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()
    parrafos = [p.strip() for p in contenido.split('*') if p.strip()]
    return parrafos

In [ ]:
# Ruta al dataset
ruta = '/content/drive/MyDrive/dataset_escuela'

In [ ]:
# Leer los archivos y asignar etiquetas
parrafos_estudiantes = leer_parrafos(ruta + '/normal.txt')
etiquetas_estudiantes = [0] * len(parrafos_estudiantes)

parrafos_docentes = leer_parrafos(ruta + '/ofensivo.txt')
etiquetas_docentes = [1] * len(parrafos_docentes)

In [ ]:
# Combinar todos los párrafos y etiquetas
todos_parrafos = parrafos_estudiantes + parrafos_docentes
todas_etiquetas = etiquetas_estudiantes + etiquetas_docentes

# Dividir en conjuntos de entrenamiento y prueba
train_texts, test_texts, train_labels, test_labels = train_test_split(
    todos_parrafos, todas_etiquetas, test_size=0.2, random_state=42
)

In [ ]:
# Definir la clase personalizada para el dataset
class TextoDataset(Dataset):
    def __init__(self, textos, etiquetas, tokenizer, max_length=512):
        self.textos = textos
        self.etiquetas = etiquetas
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        texto = self.textos[idx]
        etiqueta = self.etiquetas[idx]
        encoding = self.tokenizer(
            texto, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(etiqueta, dtype=torch.long)
        }

In [ ]:
from transformers import BertTokenizer

# Cargar el tokenizador de BERT en español uncased
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

# Crear los datasets de entrenamiento y prueba
train_dataset = TextoDataset(train_texts, train_labels, tokenizer)
test_dataset = TextoDataset(test_texts, test_labels, tokenizer)

In [ ]:
# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Cargar el modelo de BERT en español uncased para clasificación
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-uncased',
    num_labels=2
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Mover el modelo a la GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Definir el optimizador y la función de pérdida
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = CrossEntropyLoss()

In [ ]:
# Función de entrenamiento manual
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Entrenando"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) # Move labels to the device
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

In [ ]:
# Función de evaluación manual
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluando"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device) # Move labels to the device
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return total_loss / len(test_loader), accuracy

In [ ]:
# Entrenamiento y evaluación manual
num_epochs = 30
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Pérdida de entrenamiento: {train_loss:.4f}")
    print(f"Pérdida de prueba: {test_loss:.4f}, Precisión en prueba: {test_accuracy:.4f}")


Epoch 1/30


Evaluando: 100%|██████████| 4/4 [00:03<00:00,  1.33it/s]


Pérdida de entrenamiento: 0.3010
Pérdida de prueba: 0.1917, Precisión en prueba: 0.9490

Epoch 2/30


Evaluando: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


Pérdida de entrenamiento: 0.2966
Pérdida de prueba: 0.1917, Precisión en prueba: 0.9490

Epoch 3/30


...

Pérdida de entrenamiento: 0.3340
Pérdida de prueba: 0.1917, Precisión en prueba: 0.9490

Epoch 29/30


Evaluando: 100%|██████████| 4/4 [00:02<00:00,  1.37it/s]


Pérdida de entrenamiento: 0.2781
Pérdida de prueba: 0.1917, Precisión en prueba: 0.9490

Epoch 30/30


Evaluando: 100%|██████████| 4/4 [00:02<00:00,  1.37it/s]

Pérdida de entrenamiento: 0.2912
Pérdida de prueba: 0.1917, Precisión en prueba: 0.9490


In [ ]:
# Definir las clases
CLASES = {
    0: "Normal",
    1: "Ofensivo"
}
def inferir_clase(texto, model_path, tokenizer_path, max_length=512):
    """
    Función para inferir la clase de un texto dado usando un modelo BERT entrenado.

    Args:
        texto (str): El texto a clasificar.
        model_path (str): Ruta al directorio donde se guardó el modelo entrenado.
        tokenizer_path (str): Ruta al directorio donde se guardó el tokenizador.
        max_length (int): Longitud máxima de la secuencia para el tokenizador.

    Returns:
        str: La clase predicha del texto.
    """
    # Cargar el tokenizador y el modelo
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Mover el modelo a la GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenizar el texto
    encoding = tokenizer(
        texto,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Mover los tensores al dispositivo adecuado
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Realizar la inferencia
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    # Devolver la clase predicha
    return CLASES[prediccion]

In [ ]:
#Guardar el modelo entrenado (opcional)
model.save_pretrained('/content/drive/MyDrive/dataset_escuela/bert_model_manual_final')
tokenizer.save_pretrained('/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final')


('/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final/tokenizer_config.json',
 '/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final/special_tokens_map.json',
 '/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final/vocab.txt',
 '/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final/added_tokens.json')

In [ ]:
model_path = '/content/drive/MyDrive/dataset_escuela/bert_model_manual_final'
tokenizer_path = '/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final'
# Cargar el tokenizador y el modelo
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
texto_ejemplo = "Me gusta que se interesen por los temas"

clase_predicha = inferir_clase(texto_ejemplo, model_path, tokenizer_path)
print(f"El texto pertenece a la clase: {clase_predicha}")

El texto pertenece a la clase: OFENSIVO


In [ ]:
model_path = '/content/drive/MyDrive/dataset_escuela/bert_model_manual_final'
tokenizer_path = '/content/drive/MyDrive/dataset_escuela/bert_tokenizer_manual_final'

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Definir las clases
CLASES = {
    0: "NORMAL",
    1: "OFENSIVO"
}

def inferir_clase(texto, model_path, tokenizer_path, max_length=512):
    """
    Función para inferir la clase de un texto dado usando un modelo BERT entrenado.

    Args:
        texto (str): El texto a clasificar.
        model_path (str): Ruta al directorio donde se guardó el modelo entrenado.
        tokenizer_path (str): Ruta al directorio donde se guardó el tokenizador.
        max_length (int): Longitud máxima de la secuencia para el tokenizador.

    Returns:
        str: La clase predicha del texto ("NORMAL" o "OFENSIVO").
    """
    # Cargar el tokenizador y el modelo
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Mover el modelo a la GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenizar el texto
    encoding = tokenizer(
        texto,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Mover los tensores al dispositivo adecuado
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Realizar la inferencia
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # CORRECCIÓN: Usar índice [0] para obtener el primer (y único) elemento del batch
        prediccion = torch.argmax(logits, dim=1)[0].item()

    # Devolver la clase predicha
    return CLASES[prediccion]

# Función alternativa más robusta
def inferir_clase_con_probabilidades(texto, model_path, tokenizer_path, max_length=512):
    """
    Versión extendida que también devuelve las probabilidades de cada clase.
    """
    # Cargar el tokenizador y el modelo
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model = BertForSequenceClassification.from_pretrained(model_path)

    # Mover el modelo a la GPU si está disponible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Tokenizar el texto
    encoding = tokenizer(
        texto,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Mover los tensores al dispositivo adecuado
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Realizar la inferencia
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Aplicar softmax para obtener probabilidades
        probabilidades = torch.softmax(logits, dim=1)

        # Obtener la predicción
        prediccion = torch.argmax(logits, dim=1)[0].item()

        # Obtener las probabilidades individuales
        prob_normal = probabilidades[0][0].item()
        prob_ofensivo = probabilidades[0][1].item()

    return {
        'clase': CLASES[prediccion],
        'probabilidad_normal': prob_normal,
        'probabilidad_ofensivo': prob_ofensivo,
        'confianza': max(prob_normal, prob_ofensivo)
    }

# Función de prueba corregida
def probar_modelo_completo_corregido(model_path, tokenizer_path):
    """Función principal para probar el modelo con múltiples casos - VERSIÓN CORREGIDA"""

    # Configuración de rutas
    print("="*70)
    print("EVALUACIÓN COMPLETA DEL MODELO BERT CLASIFICADOR")
    print("="*70)

    # 1. CASOS BÁSICOS ESPERADOS
    print("\n1. CASOS BÁSICOS ESPERADOS")
    print("-" * 40)

    casos_normales_basicos = [
        "Hoy vamos a ver anatomía humana",
        "Abran el libro en la página veinte",
        "María, puedes leer el siguiente párrafo",
        "En el zoológico vimos un mono negro",
        "Hay una chica que lava platos en la cafetería",
        "Los estudiantes de diferentes culturas enriquecen la clase",
        "Vamos a estudiar sobre la diversidad étnica"
    ]

    casos_ofensivos_basicos = [
        "Cállate mono negro, no interrumpas",
        "Esa chica solo sirve para lavar platos",
        "Las mujeres deberían estar en la cocina",
        "Tu color de piel no te permite ser inteligente",
        "Los homosexuales no deberían estar aquí"
    ]

    print("CASOS NORMALES:")
    for texto in casos_normales_basicos:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == "NORMAL" else "✗"
            print(f"{estado} '{texto}' → {prediccion}")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")

    print("\nCASOS OFENSIVOS:")
    for texto in casos_ofensivos_basicos:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == "OFENSIVO" else "✗"
            print(f"{estado} '{texto}' → {prediccion}")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")

    # 2. CASOS ENGAÑOSOS - PALABRAS AMBIGUAS
    print("\n\n2. CASOS ENGAÑOSOS - PALABRAS AMBIGUAS")
    print("-" * 50)

    casos_engañosos = [
        # Casos con palabras que podrían ser ofensivas pero en contexto educativo
        ("En el laboratorio vamos a ver células negras y blancas", "NORMAL"),
        ("El personaje negro de la novela representa la justicia", "NORMAL"),
        ("Los puntos negros en el mapa indican las ciudades", "NORMAL"),
        ("La lista negra de palabras prohibidas en el examen", "NORMAL"),
        ("En química vemos la reacción del carbón negro", "NORMAL"),

        # Casos con referencias a género en contexto académico
        ("Las mujeres científicas han hecho grandes aportes", "NORMAL"),
        ("En la historia vemos cómo los hombres y mujeres trabajaban", "NORMAL"),
        ("La protagonista femenina del libro es muy valiente", "NORMAL"),
        ("Los roles de género han cambiado a lo largo de la historia", "NORMAL"),

        # Casos con referencias étnicas en contexto educativo
        ("Los pueblos indígenas tenían conocimientos avanzados", "NORMAL"),
        ("La cultura africana es muy rica en tradiciones", "NORMAL"),
        ("Los inmigrantes asiáticos trajeron nuevas tecnologías", "NORMAL"),
        ("La población gitana tiene una cultura nómada interesante", "NORMAL"),

        # Casos sutilmente ofensivos (más difíciles de detectar)
        ("Algunos estudiantes simplemente no tienen capacidad para esto", "OFENSIVO"),
        ("Ciertos alumnos traen problemas de su ambiente familiar", "OFENSIVO"),
        ("Esos niños de barrio no entienden estas cosas finas", "OFENSIVO"),
        ("Las niñas son naturalmente peores en matemáticas", "OFENSIVO"),
        ("Tu tipo de gente no suele destacar en ciencias", "OFENSIVO"),
    ]

    errores = 0
    for texto, esperado in casos_engañosos:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == esperado else "✗"
            if prediccion != esperado:
                errores += 1
            print(f"{estado} '{texto}' → {prediccion} (esperado: {esperado})")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")
            errores += 1

    # 3. CASOS LÍMITE - MUY SUTILES
    print("\n\n3. CASOS LÍMITE - MUY SUTILES")
    print("-" * 40)

    casos_limite = [
        # Borderline - podrían ir en cualquier dirección
        ("No todos los estudiantes tienen la misma capacidad", "NORMAL"),
        ("Algunos alumnos necesitan más apoyo que otros", "NORMAL"),
        ("Tu rendimiento no es el esperado para alguien como tú", "OFENSIVO"),
        ("Las chicas suelen ser más ordenadas que los chicos", "OFENSIVO"),
        ("Los chicos son naturalmente más agresivos", "OFENSIVO"),
        ("Tu familia probablemente no valora la educación", "OFENSIVO"),
        ("Estudiantes de tu condición social suelen abandonar", "OFENSIVO"),
        ("No esperaba mucho de ti considerando tu origen", "OFENSIVO"),
    ]

    for texto, esperado in casos_limite:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == esperado else "✗"
            if prediccion != esperado:
                errores += 1
            print(f"{estado} '{texto}' → {prediccion} (esperado: {esperado})")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")
            errores += 1

    # 4. CASOS CON NEGACIONES
    print("\n\n4. CASOS CON NEGACIONES")
    print("-" * 30)

    casos_negacion = [
        ("No vamos a discriminar a ningún estudiante", "NORMAL"),
        ("No toleraremos insultos en esta clase", "NORMAL"),
        ("No es cierto que las mujeres sean peores en matemáticas", "NORMAL"),
        ("No deberías comportarte como un mono", "OFENSIVO"),
        ("No me extraña que falles, eres de barrio", "OFENSIVO"),
        ("No esperaba nada bueno de tu tipo", "OFENSIVO"),
    ]

    for texto, esperado in casos_negacion:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == esperado else "✗"
            if prediccion != esperado:
                errores += 1
            print(f"{estado} '{texto}' → {prediccion} (esperado: {esperado})")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")
            errores += 1

    # 5. CASOS CON CONTEXTO ACADÉMICO ESPECÍFICO
    print("\n\n5. CONTEXTO ACADÉMICO ESPECÍFICO")
    print("-" * 40)

    casos_academicos = [
        # Anatomía y biología
        ("En anatomía estudiamos las diferencias entre sexos", "NORMAL"),
        ("Los caracteres sexuales secundarios aparecen en la pubertad", "NORMAL"),
        ("Las diferencias raciales son principalmente superficiales", "NORMAL"),

        # Historia y sociología
        ("La esclavitud fue una época oscura de la humanidad", "NORMAL"),
        ("Los conflictos étnicos han marcado la historia", "NORMAL"),
        ("El machismo ha sido un problema histórico", "NORMAL"),

        # Literatura
        ("En esta obra el autor usa términos despectivos de la época", "NORMAL"),
        ("El personaje expresa ideas racistas que debemos analizar", "NORMAL"),
        ("La novela critica los estereotipos de género", "NORMAL"),
    ]

    for texto in casos_academicos:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            estado = "✓" if prediccion == "NORMAL" else "✗"
            if prediccion != "NORMAL":
                errores += 1
            print(f"{estado} '{texto}' → {prediccion}")
        except Exception as e:
            print(f"❌ ERROR en '{texto}': {str(e)}")
            errores += 1

    # 6. ANÁLISIS DE MÉTRICAS
    print("\n\n6. RESUMEN DE PRUEBAS")
    print("-" * 30)

    # Calcular estadísticas básicas
    todos_los_casos = (
        [(texto, "NORMAL") for texto in casos_normales_basicos] +
        [(texto, "OFENSIVO") for texto in casos_ofensivos_basicos] +
        casos_engañosos + casos_limite + casos_negacion +
        [(texto, "NORMAL") for texto in casos_academicos]
    )

    correctos = 0
    total = len(todos_los_casos)

    for texto, esperado in todos_los_casos:
        try:
            prediccion = inferir_clase(texto, model_path, tokenizer_path)
            if prediccion == esperado:
                correctos += 1
        except Exception as e:
            print(f"Error procesando '{texto}': {str(e)}")

    precision = (correctos / total) * 100 if total > 0 else 0
    print(f"Precisión general: {correctos}/{total} ({precision:.1f}%)")
    print(f"Errores totales: {errores}")

    if precision >= 90:
        print("🎉 EXCELENTE: El modelo tiene muy buen rendimiento")
    elif precision >= 80:
        print("✅ BUENO: El modelo funciona bien, pero se puede mejorar")
    elif precision >= 70:
        print("⚠️ REGULAR: El modelo necesita más entrenamiento")
    else:
        print("❌ MALO: El modelo necesita revisión completa")

    return precision

# Prueba individual con probabilidades
def probar_texto_individual(texto, model_path, tokenizer_path):
    """Prueba un texto individual mostrando probabilidades detalladas"""
    print(f"\n🔍 ANÁLISIS DETALLADO:")
    print(f"Texto: '{texto}'")
    print("-" * 50)

    try:
        resultado = inferir_clase_con_probabilidades(texto, model_path, tokenizer_path)

        print(f"Predicción: {resultado['clase']}")
        print(f"Confianza: {resultado['confianza']:.3f}")
        print(f"Probabilidad NORMAL: {resultado['probabilidad_normal']:.3f}")
        print(f"Probabilidad OFENSIVO: {resultado['probabilidad_ofensivo']:.3f}")

        if resultado['confianza'] < 0.7:
            print("⚠️ ADVERTENCIA: Baja confianza en la predicción")
        elif resultado['confianza'] > 0.9:
            print("✅ Alta confianza en la predicción")

    except Exception as e:
        print(f"❌ ERROR: {str(e)}")


In [ ]:
probar_modelo_completo(model_path, tokenizer_path)

EVALUACIÓN COMPLETA DEL MODELO BERT CLASIFICADOR

1. CASOS BÁSICOS ESPERADOS
----------------------------------------
CASOS NORMALES:
✓ 'Hoy vamos a ver anatomía humana' → NORMAL
✓ 'Abran el libro en la página veinte' → NORMAL
✓ 'María, puedes leer el siguiente párrafo' → NORMAL
✓ 'En el zoológico vimos un mono negro' → NORMAL
✗ 'Hay una chica que lava platos en la cafetería' → OFENSIVO
✓ 'Los estudiantes de diferentes culturas enriquecen la clase' → NORMAL
✓ 'Vamos a estudiar sobre la diversidad étnica' → NORMAL

CASOS OFENSIVOS:
✓ 'Cállate mono negro, no interrumpas' → OFENSIVO
✓ 'Esa chica solo sirve para lavar platos' → OFENSIVO
✓ 'Las mujeres deberían estar en la cocina' → OFENSIVO
✓ 'Tu color de piel no te permite ser inteligente' → OFENSIVO
✓ 'Los homosexuales no deberían estar aquí' → OFENSIVO


2. CASOS ENGAÑOSOS - PALABRAS AMBIGUAS
--------------------------------------------------
✓ 'En el laboratorio vamos a ver células negras y blancas' → NORMAL (esperado: NORMAL)
✓ 'El pe

81.13207547169812

antes 81.1

# **RECONOCIMIENTO DE VOZ**

In [ ]:
## 1. Instalación de dependencias
!pip install speechbrain torch librosa pyannote.audio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install speechbrain

In [ ]:
import torch
import numpy as np
import librosa
import json
import sqlite3
from datetime import datetime
from speechbrain.pretrained import SpeakerRecognition
from collections import defaultdict
import threading
import time

class StudentParticipationSystem:
    def __init__(self, db_path="participation.db", threshold=0.7):
        """
        Sistema de control de participación estudiantil

        Args:
            db_path: Ruta de la base de datos
            threshold: Umbral de similitud para considerar coincidencia (0.7 recomendado)
        """
        self.threshold = threshold
        self.db_path = db_path

        # Determinar dispositivo
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Usando dispositivo: {self.device}")

        # Cargar modelo SpeechBrain
        self.verifier = SpeakerRecognition.from_hparams(
            source="speechbrain/spkrec-ecapa-voxceleb",
            run_opts={"device": str(self.device)}
        )

        # Diccionario para almacenar embeddings de estudiantes
        self.student_embeddings = {}
        self.student_names = {}

        # Control de tiempo para evitar múltiples registros del mismo estudiante
        self.last_detection = defaultdict(float)
        self.cooldown_time = 30  # 30 segundos entre detecciones del mismo estudiante

        # Inicializar base de datos
        self.init_database()

    def init_database(self):
        """Inicializa la base de datos SQLite"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Tabla de estudiantes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                id INTEGER PRIMARY KEY,
                name TEXT UNIQUE NOT NULL,
                embedding_path TEXT,
                total_participations INTEGER DEFAULT 0
            )
        ''')

        # Tabla de participaciones por clase
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS participations (
                id INTEGER PRIMARY KEY,
                student_id INTEGER,
                class_date DATE,
                timestamp DATETIME,
                confidence_score REAL,
                FOREIGN KEY (student_id) REFERENCES students (id)
            )
        ''')

        # Tabla de clases
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS classes (
                id INTEGER PRIMARY KEY,
                class_name TEXT,
                date DATE,
                start_time TIME,
                end_time TIME
            )
        ''')

        conn.commit()
        conn.close()

    def register_student(self, student_id, student_name, audio_samples):
        """
        Registra un nuevo estudiante con múltiples muestras de audio

        Args:
            student_id: ID único del estudiante
            student_name: Nombre del estudiante
            audio_samples: Lista de rutas de archivos de audio del estudiante
        """
        print(f"Registrando estudiante: {student_name}")

        # Procesar múltiples muestras para crear un embedding promedio
        embeddings = []

        for audio_path in audio_samples:
            try:
                # Cargar audio
                audio, sr = librosa.load(audio_path, sr=16000)
                audio_tensor = torch.from_numpy(audio).unsqueeze(0).to(self.device)

                # Extraer embedding
                embedding = self.verifier.encode_batch(audio_tensor)
                embeddings.append(embedding.squeeze().cpu().numpy())

            except Exception as e:
                print(f"Error procesando {audio_path}: {e}")
                continue

        if embeddings:
            # Promedio de embeddings para mayor robustez
            avg_embedding = np.mean(embeddings, axis=0)
            self.student_embeddings[student_id] = torch.from_numpy(avg_embedding).unsqueeze(0).to(self.device)
            self.student_names[student_id] = student_name

            # Guardar en base de datos
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            cursor.execute('''
                INSERT OR REPLACE INTO students (id, name, total_participations)
                VALUES (?, ?, 0)
            ''', (student_id, student_name))

            conn.commit()
            conn.close()

            print(f"Estudiante {student_name} registrado exitosamente")
        else:
            print(f"Error: No se pudieron procesar las muestras de audio para {student_name}")

    def load_students_from_directory(self, base_directory):
        """
        Carga estudiantes desde un directorio organizado
        Estructura esperada: base_directory/student_id/audio1.wav, audio2.wav, ...
        """
        import os

        for student_folder in os.listdir(base_directory):
            student_path = os.path.join(base_directory, student_folder)

            if os.path.isdir(student_path):
                try:
                    student_id = int(student_folder)
                    audio_files = [
                        os.path.join(student_path, f)
                        for f in os.listdir(student_path)
                        if f.endswith(('.wav', '.mp3', '.flac'))
                    ]

                    if audio_files:
                        # Usar el nombre de la carpeta como nombre del estudiante
                        # En producción, podrías cargar esto desde un archivo CSV
                        student_name = f"Estudiante_{student_id}"
                        self.register_student(student_id, student_name, audio_files[:3])  # Máximo 3 muestras

                except ValueError:
                    print(f"Ignorando carpeta con nombre inválido: {student_folder}")

    def identify_speaker(self, audio_path):
        """
        Identifica al hablante en un archivo de audio

        Args:
            audio_path: Ruta del archivo de audio a analizar

        Returns:
            tuple: (student_id, confidence_score) o (None, 0) si no se encuentra coincidencia
        """
        try:
            # Cargar audio
            audio, sr = librosa.load(audio_path, sr=16000)
            audio_tensor = torch.from_numpy(audio).unsqueeze(0).to(self.device)

            # Extraer embedding del audio de entrada
            input_embedding = self.verifier.encode_batch(audio_tensor)

            best_match = None
            best_score = 0

            # Comparar con todos los estudiantes registrados
            for student_id, student_embedding in self.student_embeddings.items():
                # Asegurar que ambos embeddings estén en el mismo dispositivo
                input_emb = input_embedding.squeeze().to(self.device)
                student_emb = student_embedding.squeeze().to(self.device)

                # Calcular similitud coseno
                similarity = torch.cosine_similarity(
                    input_emb,
                    student_emb,
                    dim=0
                ).item()

                if similarity > best_score and similarity > self.threshold:
                    best_score = similarity
                    best_match = student_id

            return best_match, best_score

        except Exception as e:
            print(f"Error identificando hablante: {e}")
            return None, 0

    def record_participation(self, student_id, confidence_score, class_date=None):
        """
        Registra una participación en la base de datos

        Args:
            student_id: ID del estudiante
            confidence_score: Puntuación de confianza
            class_date: Fecha de la clase (por defecto hoy)
        """
        if class_date is None:
            class_date = datetime.now().date()

        current_time = time.time()

        # Verificar cooldown para evitar registros duplicados
        if current_time - self.last_detection[student_id] < self.cooldown_time:
            return False

        self.last_detection[student_id] = current_time

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Registrar participación
        cursor.execute('''
            INSERT INTO participations (student_id, class_date, timestamp, confidence_score)
            VALUES (?, ?, ?, ?)
        ''', (student_id, class_date, datetime.now(), confidence_score))

        # Actualizar contador total
        cursor.execute('''
            UPDATE students
            SET total_participations = total_participations + 1
            WHERE id = ?
        ''', (student_id,))

        conn.commit()
        conn.close()

        student_name = self.student_names.get(student_id, f"ID_{student_id}")
        print(f"✓ Participación registrada: {student_name} (Confianza: {confidence_score:.3f})")

        return True

    def process_audio_stream(self, audio_path):
        """
        Procesa un archivo de audio y registra participaciones

        Args:
            audio_path: Ruta del archivo de audio a procesar
        """
        print(f"Procesando audio: {audio_path}")

        student_id, confidence = self.identify_speaker(audio_path)

        if student_id:
            self.record_participation(student_id, confidence)
        else:
            print("No se identificó ningún estudiante registrado")

    def get_participation_report(self, class_date=None):
        """
        Genera un reporte de participación

        Args:
            class_date: Fecha específica (por defecto hoy)

        Returns:
            list: Lista de tuplas (nombre, participaciones)
        """
        if class_date is None:
            class_date = datetime.now().date()

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT s.name, COUNT(p.id) as participations
            FROM students s
            LEFT JOIN participations p ON s.id = p.student_id
                AND p.class_date = ?
            GROUP BY s.id, s.name
            ORDER BY participations DESC
        ''', (class_date,))

        results = cursor.fetchall()
        conn.close()

        return results

    def print_participation_stats(self, class_date=None):
        """Imprime estadísticas de participación"""
        if class_date is None:
            class_date = datetime.now().date()

        print(f"\n=== REPORTE DE PARTICIPACIÓN - {class_date} ===")

        report = self.get_participation_report(class_date)

        for name, participations in report[:10]:  # Top 10
            print(f"{name}: {participations} participaciones")

        total_students = len(self.student_embeddings)
        active_students = sum(1 for _, p in report if p > 0)

        print(f"\nResumen:")
        print(f"- Estudiantes registrados: {total_students}")
        print(f"- Estudiantes que participaron: {active_students}")
        print(f"- Tasa de participación: {active_students/total_students*100:.1f}%")

In [ ]:
import torch
import numpy as np
import librosa
import json
import sqlite3
import pyaudio
from datetime import datetime
from resemblyzer import VoiceEncoder, preprocess_wav
from collections import defaultdict
import threading
import time
import wave
import tempfile
import os
from queue import Queue

class StudentParticipationSystem:
    def __init__(self, db_path="participation.db", threshold=0.7):
        """
        Sistema de control de participación estudiantil

        Args:
            db_path: Ruta de la base de datos
            threshold: Umbral de similitud para considerar coincidencia (0.7 recomendado)
        """
        self.threshold = threshold
        self.db_path = db_path

        # Determinar dispositivo
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Usando dispositivo: {self.device}")

        # Cargar modelo Resemblyzer
        self.encoder = VoiceEncoder()

        # Diccionario para almacenar embeddings de estudiantes
        self.student_embeddings = {}
        self.student_names = {}

        # Control de tiempo para evitar múltiples registros del mismo estudiante
        self.last_detection = defaultdict(float)
        self.cooldown_time = 30  # 30 segundos entre detecciones del mismo estudiante

        # Variables para captura de audio en tiempo real
        self.is_recording = False
        self.audio_queue = Queue()
        self.sample_rate = 16000
        self.chunk_size = 1024
        self.channels = 1

        # Inicializar base de datos
        self.init_database()

    def init_database(self):
        """Inicializa la base de datos SQLite"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Tabla de estudiantes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                id INTEGER PRIMARY KEY,
                name TEXT UNIQUE NOT NULL,
                embedding_path TEXT,
                total_participations INTEGER DEFAULT 0
            )
        ''')

        # Tabla de participaciones por clase
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS participations (
                id INTEGER PRIMARY KEY,
                student_id INTEGER,
                class_date DATE,
                timestamp DATETIME,
                confidence_score REAL,
                FOREIGN KEY (student_id) REFERENCES students (id)
            )
        ''')

        # Tabla de clases
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS classes (
                id INTEGER PRIMARY KEY,
                class_name TEXT,
                date DATE,
                start_time TIME,
                end_time TIME
            )
        ''')

        conn.commit()
        conn.close()

    def register_student(self, student_id, student_name, audio_samples):
        """
        Registra un nuevo estudiante con múltiples muestras de audio

        Args:
            student_id: ID único del estudiante
            student_name: Nombre del estudiante
            audio_samples: Lista de rutas de archivos de audio del estudiante
        """
        print(f"Registrando estudiante: {student_name}")

        # Procesar múltiples muestras para crear un embedding promedio
        embeddings = []

        for audio_path in audio_samples:
            try:
                # Cargar y preprocesar audio con Resemblyzer
                wav = preprocess_wav(audio_path)

                # Extraer embedding
                embedding = self.encoder.embed_utterance(wav)
                embeddings.append(embedding)

            except Exception as e:
                print(f"Error procesando {audio_path}: {e}")
                continue

        if embeddings:
            # Promedio de embeddings para mayor robustez
            avg_embedding = np.mean(embeddings, axis=0)
            self.student_embeddings[student_id] = avg_embedding
            self.student_names[student_id] = student_name

            # Guardar en base de datos
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            cursor.execute('''
                INSERT OR REPLACE INTO students (id, name, total_participations)
                VALUES (?, ?, 0)
            ''', (student_id, student_name))

            conn.commit()
            conn.close()

            print(f"Estudiante {student_name} registrado exitosamente")
        else:
            print(f"Error: No se pudieron procesar las muestras de audio para {student_name}")

    def load_students_from_directory(self, base_directory):
        """
        Carga estudiantes desde un directorio organizado
        Estructura esperada: base_directory/student_id/audio1.wav, audio2.wav, ...
        """
        import os

        for student_folder in os.listdir(base_directory):
            student_path = os.path.join(base_directory, student_folder)

            if os.path.isdir(student_path):
                try:
                    student_id = int(student_folder)
                    audio_files = [
                        os.path.join(student_path, f)
                        for f in os.listdir(student_path)
                        if f.endswith(('.wav', '.mp3', '.flac'))
                    ]

                    if audio_files:
                        # Usar el nombre de la carpeta como nombre del estudiante
                        # En producción, podrías cargar esto desde un archivo CSV
                        student_name = f"Estudiante_{student_id}"
                        self.register_student(student_id, student_name, audio_files[:3])  # Máximo 3 muestras

                except ValueError:
                    print(f"Ignorando carpeta con nombre inválido: {student_folder}")

    def identify_speaker(self, audio_path):
        """
        Identifica al hablante en un archivo de audio

        Args:
            audio_path: Ruta del archivo de audio a analizar

        Returns:
            tuple: (student_id, confidence_score) o (None, 0) si no se encuentra coincidencia
        """
        try:
            # Cargar y preprocesar audio con Resemblyzer
            wav = preprocess_wav(audio_path)

            # Extraer embedding del audio de entrada
            input_embedding = self.encoder.embed_utterance(wav)

            best_match = None
            best_score = 0

            # Comparar con todos los estudiantes registrados
            for student_id, student_embedding in self.student_embeddings.items():
                # Calcular similitud coseno
                similarity = np.dot(input_embedding, student_embedding) / (
                    np.linalg.norm(input_embedding) * np.linalg.norm(student_embedding)
                )

                if similarity > best_score and similarity > self.threshold:
                    best_score = similarity
                    best_match = student_id

            return best_match, best_score

        except Exception as e:
            print(f"Error identificando hablante: {e}")
            return None, 0

    def record_participation(self, student_id, confidence_score, class_date=None):
        """
        Registra una participación en la base de datos

        Args:
            student_id: ID del estudiante
            confidence_score: Puntuación de confianza
            class_date: Fecha de la clase (por defecto hoy)
        """
        if class_date is None:
            class_date = datetime.now().date()

        current_time = time.time()

        # Verificar cooldown para evitar registros duplicados
        if current_time - self.last_detection[student_id] < self.cooldown_time:
            return False

        self.last_detection[student_id] = current_time

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Registrar participación
        cursor.execute('''
            INSERT INTO participations (student_id, class_date, timestamp, confidence_score)
            VALUES (?, ?, ?, ?)
        ''', (student_id, class_date, datetime.now(), confidence_score))

        # Actualizar contador total
        cursor.execute('''
            UPDATE students
            SET total_participations = total_participations + 1
            WHERE id = ?
        ''', (student_id,))

        conn.commit()
        conn.close()

        student_name = self.student_names.get(student_id, f"ID_{student_id}")
        print(f"✓ Participación registrada: {student_name} (Confianza: {confidence_score:.3f})")

        return True

    def detect_voice_activity(self, audio_data, threshold=0.01):
        """
        Detecta si hay actividad de voz en el audio

        Args:
            audio_data: Array de numpy con datos de audio
            threshold: Umbral de energía para detectar voz

        Returns:
            bool: True si se detecta voz
        """
        # Calcular energía RMS
        rms = np.sqrt(np.mean(audio_data**2))
        return rms > threshold

    def record_audio_chunk(self, duration=3):
        """
        Graba un chunk de audio del micrófono usando Google Colab

        Args:
            duration: Duración en segundos del chunk

        Returns:
            str: Ruta del archivo temporal con el audio grabado
        """
        try:
            from google.colab import output
            from IPython.display import Javascript
            import base64
            import io

            # JavaScript para grabar audio en Colab
            RECORD = """
            const sleep = time => new Promise(resolve => setTimeout(resolve, time))
            const b2text = blob => new Promise(resolve => {
              const reader = new FileReader()
              reader.onloadend = e => resolve(e.srcElement.result)
              reader.readAsDataURL(blob)
            })

            var record = time => new Promise(async resolve => {
              stream = await navigator.mediaDevices.getUserMedia({ audio: true })
              recorder = new MediaRecorder(stream)
              chunks = []
              recorder.ondataavailable = e => chunks.push(e.data)
              recorder.start()
              await sleep(time)
              recorder.onstop = async ()=>{
                blob = new Blob(chunks)
                text = await b2text(blob)
                resolve(text)
              }
              recorder.stop()
            })
            """

            print(f"🎤 Grabando audio por {duration} segundos...")
            display(Javascript(RECORD))
            s = output.eval_js('record(%d)' % (duration * 1000))

            if not s:
                return None

            # Decodificar base64
            b = base64.b64decode(s.split(',')[1])

            # Guardar en archivo temporal
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
            temp_path = temp_file.name
            temp_file.write(b)
            temp_file.close()

            # Cargar con librosa para verificar actividad de voz
            try:
                audio_data, sr = librosa.load(temp_path, sr=self.sample_rate)

                # Verificar si hay actividad de voz
                if not self.detect_voice_activity(audio_data):
                    os.unlink(temp_path)
                    return None

                return temp_path

            except Exception as e:
                print(f"Error procesando audio: {e}")
                os.unlink(temp_path)
                return None

        except Exception as e:
            print(f"Error grabando audio: {e}")
            return None

    def start_real_time_monitoring(self, chunk_duration=3, check_interval=1):
        """
        Inicia el monitoreo en tiempo real del micrófono

        Args:
            chunk_duration: Duración de cada chunk de audio a analizar
            check_interval: Intervalo entre grabaciones en segundos
        """
        print("🚀 Iniciando monitoreo en tiempo real...")
        print("Presiona Ctrl+C para detener")

        self.is_recording = True

        try:
            while self.is_recording:
                # Grabar chunk de audio
                audio_path = self.record_audio_chunk(chunk_duration)

                if audio_path:
                    # Procesar audio para identificar estudiante
                    student_id, confidence = self.identify_speaker(audio_path)

                    if student_id:
                        student_name = self.student_names.get(student_id, f"ID_{student_id}")
                        print(f"🗣️  Detectado: {student_name} (Confianza: {confidence:.3f})")

                        # Registrar participación
                        if self.record_participation(student_id, confidence):
                            print(f"✅ Participación registrada para {student_name}")
                    else:
                        print("👤 Voz detectada pero no identificada")

                    # Limpiar archivo temporal
                    os.unlink(audio_path)
                else:
                    print("🔇 Sin actividad de voz detectada")

                # Esperar antes del siguiente chunk
                time.sleep(check_interval)

        except KeyboardInterrupt:
            print("\n⏹️  Deteniendo monitoreo...")
            self.is_recording = False
        except Exception as e:
            print(f"Error en monitoreo: {e}")
            self.is_recording = False

    def stop_monitoring(self):
        """Detiene el monitoreo en tiempo real"""
        self.is_recording = False
        print("Monitoreo detenido")

    def start_monitoring_thread(self, chunk_duration=3, check_interval=1):
        """
        Inicia el monitoreo en un hilo separado

        Args:
            chunk_duration: Duración de cada chunk de audio
            check_interval: Intervalo entre grabaciones
        """
        monitoring_thread = threading.Thread(
            target=self.start_real_time_monitoring,
            args=(chunk_duration, check_interval)
        )
        monitoring_thread.daemon = True
        monitoring_thread.start()
        return monitoring_thread

    def get_participation_report(self, class_date=None):
        """
        Genera un reporte de participación

        Args:
            class_date: Fecha específica (por defecto hoy)

        Returns:
            list: Lista de tuplas (nombre, participaciones)
        """
        if class_date is None:
            class_date = datetime.now().date()

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT s.name, COUNT(p.id) as participations
            FROM students s
            LEFT JOIN participations p ON s.id = p.student_id
                AND p.class_date = ?
            GROUP BY s.id, s.name
            ORDER BY participations DESC
        ''', (class_date,))

        results = cursor.fetchall()
        conn.close()

        return results

    def print_participation_stats(self, class_date=None):
        """Imprime estadísticas de participación"""
        if class_date is None:
            class_date = datetime.now().date()

        print(f"\n=== REPORTE DE PARTICIPACIÓN - {class_date} ===")

        report = self.get_participation_report(class_date)

        for name, participations in report[:10]:  # Top 10
            print(f"{name}: {participations} participations")

        total_students = len(self.student_embeddings)
        active_students = sum(1 for _, p in report if p > 0)

        print(f"\nResumen:")
        print(f"- Estudiantes registrados: {total_students}")
        print(f"- Estudiantes que participaron: {active_students}")
        print(f"- Tasa de participación: {active_students/total_students*100:.1f}%")

In [ ]:
# Inicializar sistema
system = StudentParticipationSystem(threshold=0.7)

Usando dispositivo: cuda
Loaded the voice encoder model on cuda in 0.02 seconds.


In [ ]:
# En Google Colab
!apt-get install -y portaudio19-dev
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (197 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
# Ejemplo de registro manual de estudiantes
system.register_student(
     student_id=5,
     student_name="Manuel Rua",
     audio_samples=["/content/drive/MyDrive/Manuel.wav"]
)

system.register_student(
     student_id=6,
     student_name="Atzel Cervantes",
     audio_samples=["/content/drive/MyDrive/Atzel.wav"]
)

system.register_student(
     student_id=7,
     student_name="Elmer Vela",
     audio_samples=["/content/drive/MyDrive/Pirlo.wav"]
)

system.register_student(
     student_id=8,
     student_name="Jorge Mendez",
     audio_samples=["/content/drive/MyDrive/Mendes.wav"]
)

    # Cargar estudiantes desde directorio (método recomendado)
system.load_students_from_directory("/content/drive/MyDrive/students_data")

Registrando estudiante: Manuel Rua
Estudiante Manuel Rua registrado exitosamente
Registrando estudiante: Atzel Cervantes
Estudiante Atzel Cervantes registrado exitosamente
Registrando estudiante: Elmer Vela
Estudiante Elmer Vela registrado exitosamente
Registrando estudiante: Jorge Mendez
Estudiante Jorge Mendez registrado exitosamente
Registrando estudiante: Estudiante_1
Estudiante Estudiante_1 registrado exitosamente
Registrando estudiante: Estudiante_3
Estudiante Estudiante_3 registrado exitosamente
Registrando estudiante: Estudiante_2
Estudiante Estudiante_2 registrado exitosamente


In [ ]:
# INICIAR MONITOREO EN TIEMPO REAL
system.start_real_time_monitoring(
    chunk_duration=6,     # Analiza cada 3 segundos
    check_interval=2    # Espera 0.5s entre análisis
)

🚀 Iniciando monitoreo en tiempo real...
Presiona Ctrl+C para detener
🎤 Grabando audio por 6 segundos...


<IPython.core.display.Javascript object>

/tmp/ipython-input-8-3341603086.py:327: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(temp_path, sr=self.sample_rate)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.11/dist-packages/resemblyzer/audio.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


🗣️  Detectado: Manuel Rua (Confianza: 0.715)
🎤 Grabando audio por 6 segundos...


<IPython.core.display.Javascript object>

...

/tmp/ipython-input-8-3341603086.py:327: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(temp_path, sr=self.sample_rate)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.11/dist-packages/resemblyzer/audio.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


👤 Voz detectada pero no identificada
🎤 Grabando audio por 6 segundos...


<IPython.core.display.Javascript object>

/tmp/ipython-input-8-3341603086.py:327: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(temp_path, sr=self.sample_rate)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.11/dist-packages/resemblyzer/audio.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


👤 Voz detectada pero no identificada

⏹️  Deteniendo monitoreo...


In [ ]:
from google.colab import output
from IPython.display import Javascript, display
import base64
import io

In [ ]:
!pip install resemblyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 76.3 MB/s eta 0:00:00
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73508 sha256=b01fdc781bf1691fbca250d3be5ced2f595e11080ed246583e3f81d51f6a26b4
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=5c9983605500517fa0dcd8d1697e52c9554486defcbc873a9cccbe811b7d0d58
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built webrtcvad typing


In [ ]:
import torch
import numpy as np
import librosa
import json
import sqlite3
from datetime import datetime
from resemblyzer import VoiceEncoder, preprocess_wav
from collections import defaultdict
import threading
import time

class StudentParticipationSystem:
    def __init__(self, db_path="participation.db", threshold=0.7):
        """
        Sistema de control de participación estudiantil

        Args:
            db_path: Ruta de la base de datos
            threshold: Umbral de similitud para considerar coincidencia (0.7 recomendado)
        """
        self.threshold = threshold
        self.db_path = db_path

        # Determinar dispositivo
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Usando dispositivo: {self.device}")

        # Cargar modelo Resemblyzer
        self.encoder = VoiceEncoder()

        # Diccionario para almacenar embeddings de estudiantes
        self.student_embeddings = {}
        self.student_names = {}

        # Control de tiempo para evitar múltiples registros del mismo estudiante
        self.last_detection = defaultdict(float)
        self.cooldown_time = 30  # 30 segundos entre detecciones del mismo estudiante

        # Inicializar base de datos
        self.init_database()

    def init_database(self):
        """Inicializa la base de datos SQLite"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Tabla de estudiantes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                id INTEGER PRIMARY KEY,
                name TEXT UNIQUE NOT NULL,
                embedding_path TEXT,
                total_participations INTEGER DEFAULT 0
            )
        ''')

        # Tabla de participaciones por clase
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS participations (
                id INTEGER PRIMARY KEY,
                student_id INTEGER,
                class_date DATE,
                timestamp DATETIME,
                confidence_score REAL,
                FOREIGN KEY (student_id) REFERENCES students (id)
            )
        ''')

        # Tabla de clases
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS classes (
                id INTEGER PRIMARY KEY,
                class_name TEXT,
                date DATE,
                start_time TIME,
                end_time TIME
            )
        ''')

        conn.commit()
        conn.close()

    def register_student(self, student_id, student_name, audio_samples):
        """
        Registra un nuevo estudiante con múltiples muestras de audio

        Args:
            student_id: ID único del estudiante
            student_name: Nombre del estudiante
            audio_samples: Lista de rutas de archivos de audio del estudiante
        """
        print(f"Registrando estudiante: {student_name}")

        # Procesar múltiples muestras para crear un embedding promedio
        embeddings = []

        for audio_path in audio_samples:
            try:
                # Cargar y preprocesar audio con Resemblyzer
                wav = preprocess_wav(audio_path)

                # Extraer embedding
                embedding = self.encoder.embed_utterance(wav)
                embeddings.append(embedding)

            except Exception as e:
                print(f"Error procesando {audio_path}: {e}")
                continue

        if embeddings:
            # Promedio de embeddings para mayor robustez
            avg_embedding = np.mean(embeddings, axis=0)
            self.student_embeddings[student_id] = avg_embedding
            self.student_names[student_id] = student_name

            # Guardar en base de datos
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            cursor.execute('''
                INSERT OR REPLACE INTO students (id, name, total_participations)
                VALUES (?, ?, 0)
            ''', (student_id, student_name))

            conn.commit()
            conn.close()

            print(f"Estudiante {student_name} registrado exitosamente")
        else:
            print(f"Error: No se pudieron procesar las muestras de audio para {student_name}")

    def load_students_from_directory(self, base_directory):
        """
        Carga estudiantes desde un directorio organizado
        Estructura esperada: base_directory/student_id/audio1.wav, audio2.wav, ...
        """
        import os

        for student_folder in os.listdir(base_directory):
            student_path = os.path.join(base_directory, student_folder)

            if os.path.isdir(student_path):
                try:
                    student_id = int(student_folder)
                    audio_files = [
                        os.path.join(student_path, f)
                        for f in os.listdir(student_path)
                        if f.endswith(('.wav', '.mp3', '.flac'))
                    ]

                    if audio_files:
                        # Usar el nombre de la carpeta como nombre del estudiante
                        # En producción, podrías cargar esto desde un archivo CSV
                        student_name = f"Estudiante_{student_id}"
                        self.register_student(student_id, student_name, audio_files[:3])  # Máximo 3 muestras

                except ValueError:
                    print(f"Ignorando carpeta con nombre inválido: {student_folder}")

    def identify_speaker(self, audio_path):
        """
        Identifica al hablante en un archivo de audio

        Args:
            audio_path: Ruta del archivo de audio a analizar

        Returns:
            tuple: (student_id, confidence_score) o (None, 0) si no se encuentra coincidencia
        """
        try:
            # Cargar y preprocesar audio con Resemblyzer
            wav = preprocess_wav(audio_path)

            # Extraer embedding del audio de entrada
            input_embedding = self.encoder.embed_utterance(wav)

            best_match = None
            best_score = 0

            # Comparar con todos los estudiantes registrados
            for student_id, student_embedding in self.student_embeddings.items():
                # Calcular similitud coseno
                similarity = np.dot(input_embedding, student_embedding) / (
                    np.linalg.norm(input_embedding) * np.linalg.norm(student_embedding)
                )

                if similarity > best_score and similarity > self.threshold:
                    best_score = similarity
                    best_match = student_id

            return best_match, best_score

        except Exception as e:
            print(f"Error identificando hablante: {e}")
            return None, 0

    def record_participation(self, student_id, confidence_score, class_date=None):
        """
        Registra una participación en la base de datos

        Args:
            student_id: ID del estudiante
            confidence_score: Puntuación de confianza
            class_date: Fecha de la clase (por defecto hoy)
        """
        if class_date is None:
            class_date = datetime.now().date()

        current_time = time.time()

        # Verificar cooldown para evitar registros duplicados
        if current_time - self.last_detection[student_id] < self.cooldown_time:
            return False

        self.last_detection[student_id] = current_time

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Registrar participación
        cursor.execute('''
            INSERT INTO participations (student_id, class_date, timestamp, confidence_score)
            VALUES (?, ?, ?, ?)
        ''', (student_id, class_date, datetime.now(), confidence_score))

        # Actualizar contador total
        cursor.execute('''
            UPDATE students
            SET total_participations = total_participations + 1
            WHERE id = ?
        ''', (student_id,))

        conn.commit()
        conn.close()

        student_name = self.student_names.get(student_id, f"ID_{student_id}")
        print(f"✓ Participación registrada: {student_name} (Confianza: {confidence_score:.3f})")

        return True

    def process_audio_stream(self, audio_path):
        """
        Procesa un archivo de audio y registra participaciones

        Args:
            audio_path: Ruta del archivo de audio a procesar
        """
        print(f"Procesando audio: {audio_path}")

        student_id, confidence = self.identify_speaker(audio_path)

        if student_id:
            self.record_participation(student_id, confidence)
        else:
            print("No se identificó ningún estudiante registrado")

    def get_participation_report(self, class_date=None):
        """
        Genera un reporte de participación

        Args:
            class_date: Fecha específica (por defecto hoy)

        Returns:
            list: Lista de tuplas (nombre, participaciones)
        """
        if class_date is None:
            class_date = datetime.now().date()

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT s.name, COUNT(p.id) as participations
            FROM students s
            LEFT JOIN participations p ON s.id = p.student_id
                AND p.class_date = ?
            GROUP BY s.id, s.name
            ORDER BY participations DESC
        ''', (class_date,))

        results = cursor.fetchall()
        conn.close()

        return results

    def print_participation_stats(self, class_date=None):
        """Imprime estadísticas de participación"""
        if class_date is None:
            class_date = datetime.now().date()

        print(f"\n=== REPORTE DE PARTICIPACIÓN - {class_date} ===")

        report = self.get_participation_report(class_date)

        for name, participations in report[:10]:  # Top 10
            print(f"{name}: {participations} participaciones")

        total_students = len(self.student_embeddings)
        active_students = sum(1 for _, p in report if p > 0)

        print(f"\nResumen:")
        print(f"- Estudiantes registrados: {total_students}")
        print(f"- Estudiantes que participaron: {active_students}")
        print(f"- Tasa de participación: {active_students/total_students*100:.1f}%")

In [ ]:
# Ejemplo de registro manual de estudiantes
system.register_student(
     student_id=5,
     student_name="Manuel Rua",
     audio_samples=["/content/drive/MyDrive/Manuel.wav"]
)

system.register_student(
     student_id=6,
     student_name="Atzel Cervantes",
     audio_samples=["/content/drive/MyDrive/Atzel.wav"]
)

system.register_student(
     student_id=7,
     student_name="Elmer Vela",
     audio_samples=["/content/drive/MyDrive/Pirlo.wav"]
)

system.register_student(
     student_id=8,
     student_name="Jorge Mendez",
     audio_samples=["/content/drive/MyDrive/Mendes.wav"]
)

# Cargar estudiantes desde directorio (método recomendado)
system.load_students_from_directory("/content/drive/MyDrive/students_data")



Registrando estudiante: Manuel Rua
Estudiante Manuel Rua registrado exitosamente
Registrando estudiante: Atzel Cervantes
Estudiante Atzel Cervantes registrado exitosamente
Registrando estudiante: Elmer Vela
Estudiante Elmer Vela registrado exitosamente
Registrando estudiante: Jorge Mendez
Estudiante Jorge Mendez registrado exitosamente
Registrando estudiante: Estudiante_1
Estudiante Estudiante_1 registrado exitosamente
Registrando estudiante: Estudiante_3
Estudiante Estudiante_3 registrado exitosamente
Registrando estudiante: Estudiante_2
Estudiante Estudiante_2 registrado exitosamente


In [ ]:
# Procesar audio en tiempo real
system.process_audio_stream("/content/drive/MyDrive/Manco.wav")

# Generar reporte
system.print_participation_stats()

print("Sistema de participación inicializado correctamente")

Procesando audio: /content/drive/MyDrive/Manco.wav

=== REPORTE DE PARTICIPACIÓN - 2025-06-28 ===
Estudiante_1: 2 participaciones
Manuel Rua: 2 participaciones
Elmer Vela: 1 participaciones
Atzel Cervantes: 0 participaciones
Estudiante_2: 0 participaciones
Estudiante_3: 0 participaciones
Jorge Mendez: 0 participaciones

Resumen:
- Estudiantes registrados: 7
- Estudiantes que participaron: 3
- Tasa de participación: 42.9%
Sistema de participación inicializado correctamente


# **Whisper**

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!apt-get install -y -qq ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
## 2) Importar y cargar el modelo Whisper
import whisper
model = whisper.load_model("base")  # o "small", "medium", "large"

In [ ]:
def _bytes_from_base64(data_url):
    header, b64 = data_url.split(",", 1)
    return base64.b64decode(b64)

def process_audio(data_url):
    # 1) decodifica y guarda el chunk .webm
    webm_bytes = _bytes_from_base64(data_url)
    tmp_webm = tempfile.NamedTemporaryFile(suffix=".webm", delete=False)
    tmp_webm.write(webm_bytes)
    tmp_webm.flush()
    tmp_webm.close()

    # 2) convierte a WAV 16 kHz mono
    tmp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    tmp_wav.close()
    cmd = [
        "ffmpeg",
        "-y",                    # sobrescribe si ya existe
        "-i", tmp_webm.name,     # input
        "-ar", "16000",          # sample rate
        "-ac", "1",              # canales mono
        tmp_wav.name
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

    # 3) transcribe con Whisper
    result = model.transcribe(tmp_wav.name, fp16=False)
    print(result["text"].strip())

    # 4) limpia archivos temporales
    os.remove(tmp_webm.name)
    os.remove(tmp_wav.name)

# registra el callback
output.register_callback('notebook.AudioCallback', process_audio)

In [ ]:
from IPython.display import Javascript, display

def start_recording(chunk_ms=3000):
    js = f"""
    (() => {{
      const rec_len = {chunk_ms};
      navigator.mediaDevices.getUserMedia({{ audio: true }})
        .then(stream => {{
          const recorder = new MediaRecorder(stream);
          recorder.ondataavailable = async e => {{
            const reader = new FileReader();
            reader.readAsDataURL(e.data);
            reader.onloadend = () => {{
              const base64data = reader.result;
              google.colab.kernel.invokeFunction('notebook.AudioCallback', [base64data], {{}});
            }};
          }};
          recorder.start(rec_len);
          recorder.onstop = () => recorder.start(rec_len);
        }})
        .catch(err => alert('Error accediendo al micrófono: ' + err));
    }})();
    """
    display(Javascript(js))


In [ ]:

# Ejecutar
start_recording(chunk_ms=3000)  # aquí defines la longitud de cada fragmento en ms

<IPython.core.display.Javascript object>

Dispositivos de audio disponibles:
Error al acceder al micrófono: [Errno -9996] Invalid input device (no default output device)
Intenta ejecutar en tu navegador y permitir acceso al micrófono


In [ ]:
# Código para transcripción en tiempo real con Whisper en Google Colab
# Ejecuta cada celda en orden

# CELDA 1: Instalación de dependencias
!pip install openai-whisper
!pip install pyaudio
!sudo apt-get install portaudio19-dev python3-pyaudio

# CELDA 2: Imports y configuración
import whisper
import pyaudio
import wave
import threading
import queue
import time
import numpy as np
from IPython.display import display, HTML, clear_output
import io

# CELDA 3: Configuración del audio
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3  # Duración de cada segmento a transcribir

# CELDA 4: Cargar modelo Whisper
print("Cargando modelo Whisper...")
model = whisper.load_model("base")  # Puedes usar "tiny", "base", "small", "medium", "large"
print("Modelo cargado exitosamente!")

# CELDA 5: Clase para manejo de audio en tiempo real
class RealTimeTranscriber:
    def __init__(self):
        self.audio_queue = queue.Queue()
        self.is_recording = False
        self.transcription_text = ""

    def audio_callback(self, in_data, frame_count, time_info, status):
        """Callback para capturar audio"""
        self.audio_queue.put(in_data)
        return (in_data, pyaudio.paContinue)

    def record_audio(self):
        """Función para grabar audio continuamente"""
        p = pyaudio.PyAudio()

        # Verificar dispositivos de audio disponibles
        print("Dispositivos de audio disponibles:")
        for i in range(p.get_device_count()):
            info = p.get_device_info_by_index(i)
            print(f"{i}: {info['name']} - Canales de entrada: {info['maxInputChannels']}")

        try:
            stream = p.open(format=FORMAT,
                          channels=CHANNELS,
                          rate=RATE,
                          input=True,
                          frames_per_buffer=CHUNK,
                          stream_callback=self.audio_callback)

            stream.start_stream()
            print("🎤 Grabación iniciada. Habla ahora...")

            while self.is_recording:
                time.sleep(0.1)

            stream.stop_stream()
            stream.close()

        except Exception as e:
            print(f"Error al acceder al micrófono: {e}")
            print("Intenta ejecutar en tu navegador y permitir acceso al micrófono")
        finally:
            p.terminate()

    def process_audio(self):
        """Procesar audio y generar transcripciones"""
        audio_buffer = []

        while self.is_recording:
            try:
                # Recopilar audio por segmentos
                if not self.audio_queue.empty():
                    data = self.audio_queue.get()
                    audio_buffer.append(data)

                    # Cuando tengamos suficiente audio (aprox 3 segundos)
                    if len(audio_buffer) >= (RATE * RECORD_SECONDS) // CHUNK:
                        # Convertir a numpy array
                        audio_data = b''.join(audio_buffer)
                        audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

                        # Transcribir con Whisper
                        try:
                            result = model.transcribe(audio_np, language='es')  # Cambia 'es' por tu idioma
                            text = result['text'].strip()

                            if text:  # Solo mostrar si hay texto
                                self.transcription_text += f"[{time.strftime('%H:%M:%S')}] {text}\n"
                                self.update_display()

                        except Exception as e:
                            print(f"Error en transcripción: {e}")

                        # Limpiar buffer (mantener overlap)
                        overlap = len(audio_buffer) // 4
                        audio_buffer = audio_buffer[-overlap:]

                time.sleep(0.1)

            except Exception as e:
                print(f"Error procesando audio: {e}")
                break

    def update_display(self):
        """Actualizar la visualización de transcripciones"""
        clear_output(wait=True)
        html_content = f"""
        <div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;
                    border: 2px solid #4CAF50; max-height: 400px; overflow-y: auto;">
            <h3 style="color: #2E7D32; margin-top: 0;">🎤 Transcripción en Tiempo Real</h3>
            <div style="font-family: monospace; white-space: pre-wrap; background-color: white;
                        padding: 15px; border-radius: 5px; border-left: 4px solid #4CAF50;">
{self.transcription_text}
            </div>
            <p style="color: #666; font-size: 12px; margin-bottom: 0;">
                💡 Presiona el botón "Detener" para finalizar la transcripción
            </p>
        </div>
        """
        display(HTML(html_content))

    def start_transcription(self):
        """Iniciar transcripción en tiempo real"""
        self.is_recording = True
        self.transcription_text = ""

        # Iniciar hilos para grabación y procesamiento
        record_thread = threading.Thread(target=self.record_audio)
        process_thread = threading.Thread(target=self.process_audio)

        record_thread.daemon = True
        process_thread.daemon = True

        record_thread.start()
        process_thread.start()

        return record_thread, process_thread

    def stop_transcription(self):
        """Detener transcripción"""
        self.is_recording = False
        print("🛑 Transcripción detenida.")

# CELDA 6: Inicializar transcriptor
transcriber = RealTimeTranscriber()

# CELDA 7: Funciones de control
def start_recording():
    """Iniciar grabación y transcripción"""
    print("🚀 Iniciando transcripción en tiempo real...")
    threads = transcriber.start_transcription()
    return threads

def stop_recording():
    """Detener grabación y transcripción"""
    transcriber.stop_transcription()

# CELDA 8: Controles principales
print("=" * 60)
print("🎙️  WHISPER TRANSCRIPCIÓN EN TIEMPO REAL")
print("=" * 60)
print()
print("📋 INSTRUCCIONES:")
print("1. Ejecuta start_recording() para comenzar")
print("2. Habla cerca del micrófono")
print("3. Ejecuta stop_recording() para detener")
print()
print("⚙️  CONFIGURACIÓN ACTUAL:")
print(f"   • Modelo: {model.__class__.__name__}")
print(f"   • Idioma: Español (cambia en process_audio si necesitas otro)")
print(f"   • Duración de segmentos: {RECORD_SECONDS} segundos")
print(f"   • Frecuencia de muestreo: {RATE} Hz")
print()
print("🔧 COMANDOS:")
print("   • Para iniciar: start_recording()")
print("   • Para detener: stop_recording()")
print()

# CELDA 9: Ejemplo de uso
# Descomenta las siguientes líneas para iniciar automáticamente:
# print("Iniciando en 3 segundos...")
# time.sleep(3)
# threads = start_recording()

print("✅ Todo listo! Ejecuta start_recording() cuando quieras comenzar.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  python3-pyaudio
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 25.9 kB of archives.
After this operation, 117 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 25.9 kB in 0s (112 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable 

In [ ]:
# Para iniciar la transcripción:
threads = start_recording()


🚀 Iniciando transcripción en tiempo real...


In [ ]:

# Para detener:
stop_recording()

🛑 Transcripción detenida.


In [ ]:
# SOLUCIÓN PARA WHISPER EN COLAB - Grabación por chunks con interfaz web
# Ejecuta cada celda en orden

# CELDA 1: Instalación de dependencias
!pip install openai-whisper
!pip install gradio
!apt-get install ffmpeg

# CELDA 2: Imports
import whisper
import gradio as gr
import numpy as np
import tempfile
import os
from datetime import datetime
import threading
import time

# CELDA 3: Cargar modelo Whisper
print("🔄 Cargando modelo Whisper...")
model = whisper.load_model("base")  # Cambia por "small", "medium", "large" si quieres más precisión
print("✅ Modelo Whisper cargado!")

# CELDA 4: Variables globales para transcripción continua
transcription_history = []
is_running = False

def transcribe_audio(audio_file):
    """Transcribir archivo de audio individual"""
    if audio_file is None:
        return "❌ No se detectó audio"

    try:
        # Transcribir con Whisper
        result = model.transcribe(audio_file, language='es')  # Cambia 'es' por tu idioma
        text = result['text'].strip()

        if text:
            timestamp = datetime.now().strftime("%H:%M:%S")
            transcription_entry = f"[{timestamp}] {text}"
            transcription_history.append(transcription_entry)

            # Mantener solo las últimas 50 transcripciones
            if len(transcription_history) > 50:
                transcription_history.pop(0)

            return "\n".join(transcription_history)
        else:
            return "\n".join(transcription_history) + "\n⚠️ No se detectó voz clara"

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        transcription_history.append(error_msg)
        return "\n".join(transcription_history)

def clear_history():
    """Limpiar historial de transcripciones"""
    global transcription_history
    transcription_history = []
    return "🧹 Historial limpiado - Listo para nuevas transcripciones"

# CELDA 5: Crear interfaz con Gradio
print("🎯 Creando interfaz web...")

# Interfaz principal
with gr.Blocks(title="🎤 Whisper Transcripción", theme=gr.themes.Soft()) as interface:
    gr.Markdown("""
    # 🎤 Whisper - Transcripción de Voz en Tiempo Real

    ## 📋 Instrucciones:
    1. **Haz clic en el micrófono** 🎙️ para empezar a grabar
    2. **Habla claramente** durante 3-10 segundos
    3. **Detén la grabación** y espera la transcripción
    4. **Repite** para continuar transcribiendo

    ### 💡 Consejos:
    - Habla cerca del micrófono
    - Evita ruido de fondo
    - Grabaciones de 5-10 segundos funcionan mejor
    """)

    with gr.Row():
        with gr.Column(scale=2):
            # Input de audio con grabación
            audio_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="🎙️ Graba tu voz aquí",
                show_download_button=False
            )

            with gr.Row():
                transcribe_btn = gr.Button("🔄 Transcribir", variant="primary", size="lg")
                clear_btn = gr.Button("🧹 Limpiar Historial", variant="secondary")

        with gr.Column(scale=3):
            # Output de transcripción
            output_text = gr.Textbox(
                label="📝 Transcripciones",
                lines=15,
                max_lines=20,
                placeholder="Las transcripciones aparecerán aquí...",
                show_copy_button=True
            )

    # Información adicional
    gr.Markdown("""
    ### ⚙️ Configuración actual:
    - **Modelo**: Whisper Base (buen balance velocidad/precisión)
    - **Idioma**: Español (configurable)
    - **Formato**: Automático

    ### 🔧 Para cambiar idioma:
    Modifica `language='es'` en el código por:
    - `'en'` para inglés
    - `'fr'` para francés
    - `'de'` para alemán
    - `None` para detección automática
    """)

    # Eventos
    transcribe_btn.click(
        fn=transcribe_audio,
        inputs=[audio_input],
        outputs=[output_text]
    )

    clear_btn.click(
        fn=clear_history,
        outputs=[output_text]
    )

    # Auto-transcribir cuando se grabe algo
    audio_input.change(
        fn=transcribe_audio,
        inputs=[audio_input],
        outputs=[output_text]
    )

# CELDA 6: Lanzar la interfaz
print("🚀 Iniciando interfaz web...")
print("=" * 60)
print("🎉 ¡INTERFAZ LISTA!")
print("📱 Se abrirá una ventana web donde podrás:")
print("   • Grabar con el micrófono")
print("   • Ver transcripciones en tiempo real")
print("   • Copiar el texto transcrito")
print("=" * 60)

# Lanzar con configuración optimizada para Colab
interface.launch(
    share=True,          # Crear enlace público
    debug=False,         # Sin debug para mejor rendimiento
    server_name="0.0.0.0",  # Accesible desde cualquier IP
    server_port=7860,    # Puerto estándar
    show_error=True,     # Mostrar errores
    quiet=False          # Mostrar logs
)

# CELDA 7: Versión simplificada si la anterior no funciona
print("\n" + "="*50)
print("🔄 VERSIÓN ALTERNATIVA SIMPLE")
print("="*50)

def simple_transcribe():
    """Versión simple - un archivo a la vez"""
    print("📁 Sube un archivo de audio o usa la grabadora web:")

    def process_file(audio_file):
        if audio_file is None:
            return "Sube un archivo de audio"

        result = model.transcribe(audio_file, language='es')
        return result['text']

    simple_interface = gr.Interface(
        fn=process_file,
        inputs=gr.Audio(sources=["microphone", "upload"], type="filepath"),
        outputs=gr.Textbox(label="Transcripción", lines=5),
        title="🎤 Whisper Simple",
        description="Graba o sube audio para transcribir"
    )

    return simple_interface

# Para usar la versión simple, ejecuta:
# simple_interface = simple_transcribe()
# simple_interface.launch(share=True)

print("✅ Todo configurado!")
print("\n🔧 Si tienes problemas:")
print("1. Usa la interfaz web que se abrió arriba")
print("2. O ejecuta la versión simple al final")
print("3. Asegúrate de permitir acceso al micrófono en el navegador")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
🔄 Cargando modelo Whisper...
✅ Modelo Whisper cargado!
🎯 Creando interfaz web...
🚀 Iniciando interfaz web...
🎉 ¡INTERFAZ LISTA!
📱 Se abrirá una ventana web donde podrás:
   • Grabar con el micrófono
   • Ver transcripciones en tiempo real
   • Copiar el texto transcrito
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f6ec59f08be6f45a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔄 VERSIÓN ALTERNATIVA SIMPLE
✅ Todo configurado!

🔧 Si tienes problemas:
1. Usa la interfaz web que se abrió arriba
2. O ejecuta la versión simple al final
3. Asegúrate de permitir acceso al micrófono en el navegador


In [ ]:
simple_interface = simple_transcribe()
simple_interface.launch(share=True)

📁 Sube un archivo de audio o usa la grabadora web:
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f40fc0708f1b7f00d6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# WHISPER SIMPLE PARA COLAB - Solo Terminal
# Ejecuta esta celda y luego usa las funciones

!pip install openai-whisper -q

import whisper
import base64
from IPython.display import HTML, display, Javascript
from google.colab.output import eval_js
import io
import wave
import numpy as np
from datetime import datetime

# Cargar modelo
print("🔄 Cargando Whisper...")
model = whisper.load_model("base")
print("✅ Whisper listo!")

def grabar_y_transcribir(segundos=5):
    """
    Graba audio del micrófono y lo transcribe
    """
    print(f"🎙️ Preparando grabación de {segundos} segundos...")

    # JavaScript para grabación
    js_code = f"""
    const sleep = time => new Promise(resolve => setTimeout(resolve, time))

    const record = async (time) => {{
        const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }})
        const recorder = new MediaRecorder(stream)
        const chunks = []

        recorder.ondataavailable = e => chunks.push(e.data)

        const audioPromise = new Promise(resolve => {{
            recorder.onstop = () => {{
                const blob = new Blob(chunks, {{ type: 'audio/wav' }})
                const reader = new FileReader()
                reader.onloadend = () => resolve(reader.result.split(',')[1])
                reader.readAsDataURL(blob)
            }}
        }})

        recorder.start()
        console.log('🔴 Grabando...')
        await sleep(time)
        recorder.stop()
        stream.getTracks().forEach(track => track.stop())

        return await audioPromise
    }}

    record({segundos * 1000})
    """

    print(f"🔴 Grabando {segundos} segundos... ¡HABLA AHORA!")

    try:
        # Ejecutar JavaScript y obtener audio
        audio_data = eval_js(js_code)

        if audio_data:
            # Decodificar y procesar audio
            audio_bytes = base64.b64decode(audio_data)

            # Crear archivo temporal
            with open('/tmp/audio.wav', 'wb') as f:
                f.write(audio_bytes)

            print("✅ Audio capturado, transcribiendo...")

            # Transcribir con Whisper
            result = model.transcribe('/tmp/audio.wav', language='es')

            # Mostrar resultado
            timestamp = datetime.now().strftime("%H:%M:%S")
            text = result['text'].strip()

            print("\n" + "="*50)
            print(f"📝 TRANSCRIPCIÓN [{timestamp}]:")
            print("="*50)
            print(f"🗣️  {text}")
            print("="*50)

            return text
        else:
            print("❌ No se pudo capturar audio")
            return None

    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Asegúrate de permitir acceso al micrófono")
        return None

def transcribir_continuo():
    """
    Modo continuo - transcribe múltiples grabaciones
    """
    print("\n🔄 MODO CONTINUO ACTIVADO")
    print("="*40)
    print("🎯 Instrucciones:")
    print("  • Cada grabación dura 5 segundos")
    print("  • Habla cuando veas 'HABLA AHORA!'")
    print("  • Presiona Enter para continuar")
    print("  • Escribe 'q' para salir")
    print("="*40)

    transcripciones = []
    contador = 1

    while True:
        print(f"\n📍 Grabación #{contador}")
        input("⏸️  Presiona Enter para grabar (o 'q' para salir): ")

        if input == 'q':
            break

        texto = grabar_y_transcribir(5)
        if texto:
            transcripciones.append(f"[{contador}] {texto}")

        contador += 1

        # Mostrar todas las transcripciones
        if transcripciones:
            print(f"\n📋 HISTORIAL ({len(transcripciones)} grabaciones):")
            print("-" * 40)
            for t in transcripciones[-5:]:  # Últimas 5
                print(f"  {t}")
            if len(transcripciones) > 5:
                print(f"  ... y {len(transcripciones)-5} más")

    print("✅ Modo continuo terminado")
    return transcripciones

# FUNCIONES LISTAS PARA USAR:
print("\n🎯 FUNCIONES DISPONIBLES:")
print("="*40)
print("1️⃣  grabar_y_transcribir(5)     # Graba 5 segundos")
print("2️⃣  transcribir_continuo()      # Modo múltiples grabaciones")
print("="*40)
print()
print("💡 EJEMPLOS DE USO:")
print("   grabar_y_transcribir(3)   # Grabación corta")
print("   grabar_y_transcribir(10)  # Grabación larga")
print("   transcribir_continuo()    # Sesión completa")
print()
print("✅ ¡Todo listo! Usa las funciones arriba")

🔄 Cargando Whisper...
✅ Whisper listo!

🎯 FUNCIONES DISPONIBLES:
1️⃣  grabar_y_transcribir(5)     # Graba 5 segundos
2️⃣  transcribir_continuo()      # Modo múltiples grabaciones

💡 EJEMPLOS DE USO:
   grabar_y_transcribir(3)   # Grabación corta
   grabar_y_transcribir(10)  # Grabación larga
   transcribir_continuo()    # Sesión completa

✅ ¡Todo listo! Usa las funciones arriba


In [ ]:
# Modo continuo (múltiples grabaciones)
transcribir_continuo()


🔄 MODO CONTINUO ACTIVADO
🎯 Instrucciones:
  • Cada grabación dura 5 segundos
  • Habla cuando veas 'HABLA AHORA!'
  • Presiona Enter para continuar
  • Escribe 'q' para salir

📍 Grabación #1
🎙️ Preparando grabación de 5 segundos...
🔴 Grabando 5 segundos... ¡HABLA AHORA!
✅ Audio capturado, transcribiendo...

📝 TRANSCRIPCIÓN [08:46:40]:
🗣️  Bueno, empezar a transcribir, por favor transcriba.

📋 HISTORIAL (1 grabaciones):
----------------------------------------
  [1] Bueno, empezar a transcribir, por favor transcriba.

📍 Grabación #2
🎙️ Preparando grabación de 5 segundos...
🔴 Grabando 5 segundos... ¡HABLA AHORA!
✅ Audio capturado, transcribiendo...

📝 TRANSCRIPCIÓN [08:46:55]:
🗣️  buenas noches empiezas a transmitir el video y la duración de la

📋 HISTORIAL (2 grabaciones):
----------------------------------------
  [1] Bueno, empezar a transcribir, por favor transcriba.
  [2] buenas noches empiezas a transmitir el video y la duración de la

📍 Grabación #3
🎙️ Preparando grabación de 5 s

In [ ]:
# Intentar versión completa primero
whisper_rt = RealTimeWhisper()
whisper_rt.start()
print("\n⚠️  La versión de terminal completa no funciona en Colab")
print("🔄 Cambiando a versión manual...")
record_func = colab_version()

# Auto-ejecutar una grabación de prueba
print("\n🧪 Ejecutando grabación de prueba...")
record_func(3)

🔄 Cargando Whisper...
✅ Whisper cargado!

📱 Dispositivos de audio:
❌ Error con micrófono: [Errno -9996] Invalid input device (no default output device)

💡 Soluciones:
1. Ejecuta en tu máquina local (no Colab)
2. O usa la versión alternativa abajo
✅ Whisper terminado

⚠️  La versión de terminal completa no funciona en Colab
🔄 Cambiando a versión manual...

🔄 VERSIÓN COLAB (grabación manual)
🎙️ Usa esta función para grabar:
record_audio(5)  # Graba 5 segundos

🧪 Ejecutando grabación de prueba...
